In [ ]:
# Libraries for data processing and visualization
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # Visualization
import plotly.express as px # Visualization
import seaborn as sns # Visualization

# Libraries for preprocessing, model building, and MLflow
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
from mlflow.models.signature import infer_signature

# For Evalution we will use these library

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, confusion_matrix, 
    roc_curve, roc_auc_score, auc, precision_recall_curve
)

In [ ]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/Users/kittipot/Desktop/Credit_card_data/creditcard.csv')

In [ ]:
# Inspect dataset
print(credit_card_data.head())
print(credit_card_data.tail())
# dataset informations
print(credit_card_data.info())

The dataset was retrieved from an open-source website, Kaggle.com. It contains data on transactions made in 2013 by European credit card users in two days only. Thedataset consists of 31 attributes and 284,808 rows. Twenty-eight attributes are numeric variables that, due to the confidentiality and privacy of the customers, have been transformed using PCA transformation; the three remaining attributes are ”Time”, which contains the elapsed seconds between the first and other transactions of each Attribute, ”Amount” is the amount of each transaction, and the final attribute “Class” which contains binary variableswhere “1” is a case of fraudulent transaction, and “0” is not as case of fraudulent transaction.

In [ ]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

In [ ]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

In [ ]:
# drop "Time" Columns
credit_card_data = credit_card_data.drop("Time", axis=1)

โดยทั่วไป Time ถูกลบออกเพราะมันไม่ได้มีส่วนช่วยที่ชัดเจนในการจำแนกการโกง และอาจเพิ่มความซับซ้อนโดยไม่จำเป็น.

In [ ]:
# Convert datatype from object to numeric
print(credit_card_data['Amount'].dtype)
credit_card_data['Amount'] = pd.to_numeric(credit_card_data['Amount'], errors='coerce')

print(credit_card_data['Amount'].describe())
print(credit_card_data['Amount'].unique())

errors='coerce' ถูกใช้เพื่อป้องกันข้อผิดพลาดเมื่อแปลงข้อมูล และช่วยจัดการกับข้อมูลที่ไม่สามารถแปลงเป็นตัวเลขได้อย่างเหมาะสมโดยแปลงเป็น NaN.
.unique() ใช้เพื่อดึง ค่าที่ไม่ซ้ำกัน ในคอลัมน์หรือ Series ซึ่งช่วยให้สามารถตรวจสอบข้อมูลที่ซ้ำซ้อนและค่าที่ผิดปกติได้ง่าย.

In [ ]:
# Create instance
scaler = StandardScaler()

# Normalizaing data
credit_card_data['std_Amount'] = scaler.fit_transform(credit_card_data['Amount'].values.reshape (-1,1))

# remove "Amount" Columns
credit_card_data = credit_card_data.drop("Amount", axis=1)

เลือกใช้ StandardScaler เนื่องจากเราจะใช้อัลกอริธึมที่อ่อนไหวต่อขนาดของฟีเจอร์ (SVM)

In [ ]:
import imblearn 
from imblearn.under_sampling import RandomUnderSampler 

undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)

เลือกใช้ RandomUnderSampler เนื่องจาก 
1.แก้ปัญหา Class Imbalance:
ในปัญหา class imbalance ข้อมูลในคลาสที่มีจำนวนมากกว่าจะครอบงำโมเดล ส่งผลให้โมเดลให้ความสำคัญกับคลาสที่มีตัวอย่างเยอะ และละเลยคลาสที่มีตัวอย่างน้อย
RandomUnderSampler ลดจำนวนตัวอย่างในคลาสที่มีมากเกินไป (majority class) ให้เท่ากับคลาสที่มีน้อยกว่า (minority class) เพื่อให้เกิดความสมดุล
2.เพิ่มประสิทธิภาพโมเดล:
โมเดลที่เทรนกับชุดข้อมูลที่สมดุลมักจะมีประสิทธิภาพดีกว่าในแง่ของเมตริก เช่น F1 Score, Recall, Precision โดยเฉพาะสำหรับคลาสที่มีจำนวนน้อย
3.ลดเวลาในการประมวลผล:
เมื่อคลาสที่มีจำนวนมากถูกลดขนาดลง จะลดขนาดของชุดข้อมูลโดยรวม ทำให้การเทรนโมเดลเร็วขึ้นและใช้ทรัพยากรน้อยลง
4.การทำงานง่ายและตรงไปตรงมา:
RandomUnderSampler เป็นวิธีที่ง่ายและเร็วที่สุดในการสร้างความสมดุล โดยเพียงแค่เลือกตัวอย่างจาก majority class แบบสุ่มจนกระทั่งจำนวนตัวอย่างในทุกคลาสเท่ากัน
ไม่มีพารามิเตอร์ที่ซับซ้อนในการตั้งค่า

In [ ]:
# Define Feature & Target
X = credit_card_data.drop(columns=["Class"])
Y = credit_card_data["Class"]

# Undersample
X_under, Y_under = undersample.fit_resample(X, Y)
test = pd.DataFrame(Y_under, columns = ['Class'])

In [ ]:
# Visualizing undersampling results
fig, axs = plt.subplots(ncols=2, figsize=(13,4.5))
sns.countplot(x="Class", data=credit_card_data, ax=axs[0])
sns.countplot(x="Class", data=test, ax=axs[1])

fig.suptitle("Class repartition before and after undersampling")
a1=fig.axes[0]
a1.set_title("Before")
a2=fig.axes[1]
a2.set_title("After")

In [ ]:
# Split data into train & test set
X_train, X_test, y_train, y_test = train_test_split(X_under, Y_under, test_size=0.2, random_state=1)

In [ ]:
# Use mlflow manage model life cycle
with mlflow.start_run():
    
    # Create & train model
    model = SVC(probability=True, random_state=2)
    model.fit(X_train, y_train)

    # predict results
    y_pred = model.predict(X_test)

    # calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Save parameters & metrics in MLflow
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)

    # Create input_example
    input_example = X_test.iloc[:5]  # Use 5 rows from X_test

    # Create signature
    signature = infer_signature(X_test, model.predict(X_test))

    # Save trained model in MLflow
    mlflow.sklearn.log_model(model, "SVC_Model", input_example=input_example, signature=signature)

    print(f"Model logged with accuracy: {accuracy}")
    print(f"Model logged with precision: {precision}")
    print(f"Model logged with recall: {recall}")

1. Accuracy: 0.9459 (~94.6%)
Accuracy คือสัดส่วนของจำนวนการทำนายที่ถูกต้อง (ทั้งคลาสโกงและไม่โกง) ต่อจำนวนตัวอย่างทั้งหมดในชุดข้อมูล.

ค่า 0.9459 หมายความว่า โมเดลสามารถทำนายได้ถูกต้องประมาณ 94.6% ของข้อมูลทั้งหมด.
อย่างไรก็ตาม Accuracy อาจไม่ใช่ตัวชี้วัดที่ดีที่สุดในกรณีที่ชุดข้อมูลมีการกระจายของคลาสไม่สมดุล (เช่น มีธุรกรรมที่ไม่โกงมากกว่าธุรกรรมที่โกง) ซึ่งอาจทำให้โมเดลมีการทำนายคลาสที่ไม่โกงได้ดีมากกว่า.

2. Precision: 0.9894 (~98.9%)
Precision คือความแม่นยำของโมเดลในการทำนายคลาส "โกง" (positive class)
ค่า 0.9894 หมายความว่า 98.9% ของธุรกรรมที่โมเดลทำนายว่าเป็น "โกง" เป็นการทำนายที่ถูกต้อง (จริง ๆ แล้วเป็นการโกง).
การมีค่า precision ที่สูงเช่นนี้หมายความว่าโมเดลไม่ค่อยทำนายว่าธุรกรรมที่ไม่โกงเป็นการโกง (false positives ต่ำ).

3. Recall: 0.8611 (~86.1%)
Recall คือความสามารถของโมเดลในการระบุธุรกรรมที่โกงทั้งหมด (True Positives) จากทุก ๆ ธุรกรรมที่โกง (รวมทั้ง False Negatives). 
ค่า 0.8611 หมายความว่าโมเดลสามารถทำนายได้ 86.1% ของธุรกรรมที่โกง (แต่ยังมีบางธุรกรรมที่โกงไม่ได้รับการทำนายเป็นการโกง).
Recall ที่สูงเป็นการบ่งชี้ว่าโมเดลมีประสิทธิภาพในการค้นหาธุรกรรมที่โกง (ไม่ทำนายผิดเป็นคลาสที่ไม่โกง).

In [ ]:
# Create confusion matrix
matrix_svm = confusion_matrix(y_test, y_pred)
# Create confusion matrix dataframe
cm_svm = pd.DataFrame(matrix_svm, index=['not_fraud', 'fraud'], columns=['not_fraud', 'fraud'])

# Visualize confusion matrix by heatmap
sns.heatmap(cm_svm, annot=True, cbar=None, cmap="Blues", fmt = 'g')
plt.title("Confusion Matrix SVM"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")
plt.show()

จากกราฟจะเห็นว่า
TP = 187 FN = 1           TP:เป็นธุรกรรมไม่โกงและโมเดลทำนายเป็นไม่โกง        FN:เป็นธุรกรรมไม่โกงและโมเดลทำนายเป็นโกง 
FP = 15  TN = 93          FP: เป็นธุรกรรมโกงและโมเดลทำนายเป็นไม่โกง         TN:เป็นธุรกรรมโกงและโมเดลทำนายเป็นโกง 

ต่อไปเราจะทำการคำนวณค่าAUCของModel SVM โดยใช้ ROC Curve ซึ่งเป็นเครื่องมือวัดประสิทธิภาพของโมเดลการทำนายแบบสองคลาส (binary classification) 

In [ ]:
# Find fpr,tpr & AUC
y_pred_svm_proba = model.predict_proba(X_test)[::,1]
fpr_svm, tpr_svm, _ = roc_curve(y_test,  y_pred_svm_proba)
auc_svm = roc_auc_score(y_test, y_pred_svm_proba)
print("AUC SVM :", auc_svm)

1.y_pred_svm_proba = model.predict_proba(X_test)[::,1]
การคำนวณ: ใช้เมธอด predict_proba จากโมเดล SVM เพื่อคาดการณ์ความน่าจะเป็น (probability) ของแต่ละคลาสสำหรับข้อมูลทดสอบ (X_test):
model.predict_proba(X_test) คืนค่าเป็น array สองมิติ ที่แสดงความน่าจะเป็นของคลาสที่เป็น 0 และ 1
[::,1] เลือกเฉพาะค่าความน่าจะเป็นของคลาส 1 (การฉ้อโกง)
ผลลัพธ์: y_pred_svm_proba จะเป็น array ที่มีค่าความน่าจะเป็นของคลาส 1 สำหรับข้อมูลทดสอบ
2.fpr_svm, tpr_svm, _ = metrics.roc_curve(y_test, y_pred_svm_proba)
การคำนวณ: ใช้ฟังก์ชัน roc_curve จากไลบรารี sklearn.metrics เพื่อสร้างค่า:
tpr_svm (True Positive Rate):  TP/TP+FN
fpr_svm (False Positive Rate): FP/FP+TN
3.auc_svm = metrics.roc_auc_score(y_test, y_pred_svm_proba)
การคำนวณ: ใช้ฟังก์ชัน roc_auc_score จาก sklearn.metrics เพื่อคำนวณค่าพื้นที่ใต้โค้ง ROC (AUC - Area Under the Curve)
AUC เป็นตัวชี้วัดประสิทธิภาพของโมเดล
ค่า AUC จะอยู่ในช่วง 0 ถึง 1
1.0 หมายถึงโมเดลสมบูรณ์แบบ
0.5 หมายถึงโมเดลแย่เท่าการสุ่มเดา

ROC (Receiver Operating Characteristic) และ AUC (Area Under the Curve) เป็นเครื่องมือที่มีความสัมพันธ์กันในการประเมินประสิทธิภาพของโมเดลการจำแนกประเภท (classification)
ความสัมพันธ์ระหว่าง ROC และ AUC:
ROC Curve แสดง การกระจายตัว ของ TPR และ FPR ภายใต้ค่า threshold ที่เปลี่ยนแปลง
AUC คือ การสรุปผล ของ ROC Curve ในรูปของตัวเลขเดียว เพื่อบ่งบอกประสิทธิภาพของโมเดลในทุก ๆ threshold
กล่าวง่าย ๆ:
ROC คือกราฟแสดงการวัดประสิทธิภาพ
AUC คือค่าตัวเลขที่ได้จาก ROC เพื่อวัดว่าโมเดลดีแค่ไหน

In [ ]:
# Visualize ROC Curve
plt.plot(fpr_svm,tpr_svm,label="SVM, auc={:.3f})".format(auc_svm))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('SVM ROC curve')
plt.legend(loc=4)
plt.show()

1.plt.plot(fpr_svm, tpr_svm, label="SVM, auc={:.3f})".format(auc_svm))
การทำงาน:
สร้างกราฟ ROC Curve โดยใช้ค่า False Positive Rate (FPR) และ True Positive Rate (TPR) ของโมเดล SVM
ใช้ label เพื่อแสดงคำอธิบายกราฟ โดยรวมค่าคำนวณ AUC (พื้นที่ใต้โค้ง ROC) ลงไปในรูปแบบตัวเลข 3 ตำแหน่งทศนิยม ({:.3f})
ผลลัพธ์:
เส้น ROC Curve ที่มีป้ายกำกับ เช่น "SVM, auc=0.976"
2.plt.plot([0, 1], [0, 1], 'k--')
การทำงาน:
เพิ่มเส้นทแยงมุมเส้นประ ('k--' หมายถึงเส้นสีดำและแบบ dashed line) ซึ่งแทนกราฟของโมเดลที่ไม่มีประสิทธิภาพ (AUC = 0.5)
เส้นนี้เริ่มจาก (0,0) ถึง (1,1) ซึ่งแสดงว่าค่า FPR และ TPR เพิ่มขึ้นพร้อมกัน
ผลลัพธ์:
เส้นทแยงมุมเพื่อใช้เป็นเกณฑ์เปรียบเทียบกับ ROC Curve ของโมเดล
3.plt.legend(loc=4)
การทำงาน:
แสดงป้ายคำอธิบายกราฟ (legend) โดยตำแหน่งของป้ายกำกับถูกตั้งค่าเป็น loc=4 ซึ่งหมายถึงมุมขวาล่างของกราฟ
ผลลัพธ์ที่ได้จากโค้ด
กราฟ ROC Curve ของโมเดล SVM จะถูกสร้างขึ้น
มีเส้น ROC ที่บ่งบอกประสิทธิภาพของโมเดล และมี AUC แสดงในป้ายกำกับ
เส้นทแยงมุมเส้นประใช้เปรียบเทียบกับเส้น ROC ของโมเดล

In [ ]:
# Find Precision & recall
svm_precision, svm_recall, _ = precision_recall_curve(y_test, y_pred_svm_proba)
no_skill = len(y_test[y_test==1]) / len(y_test) # Find Precision of no skill model
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', color='black', label='No Skill')
plt.plot(svm_recall, svm_precision, color='orange', label='SVM')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend()
plt.show()

การใช้ No Skill Model
การเปรียบเทียบประสิทธิภาพของโมเดล:
ใช้ No Skill Model เป็นตัวอ้างอิงพื้นฐาน (baseline) เพื่อประเมินประสิทธิภาพของโมเดลที่เราพัฒนาขึ้น เช่น
หากโมเดลที่พัฒนาใหม่ทำงานไม่ดีเกิน No Skill Model แสดงว่าโมเดลของเรายังไม่ดีพอ
หากโมเดลที่พัฒนาใหม่ทำงานดีกว่า No Skill Model แสดงว่าโมเดลมีความสามารถในการแยกแยะข้อมูลได้
การตรวจสอบการทำงานของโมเดล:
เมื่อใช้ No Skill Model เป็นจุดเริ่มต้น เปรียบเทียบกับโมเดลจริงจะช่วยให้เห็นได้ชัดว่าประสิทธิภาพของโมเดลที่พัฒนามานั้นเพิ่มขึ้นมากแค่ไหน

ยิ่ง Precision และ Recall สูงมากขึ้น (เส้นโค้งเข้าใกล้มุมขวาบน) หมายถึงโมเดลมีประสิทธิภาพสูง